# **I. Perkenalan**

=================================================

### **Milestone 3 (Great Expectation)**

##### Nama  : Achmed Alridho Zulkarnaen

##### Batch : FTDS-037-RMT

##### File ipynb ini dibuat untuk melakukan validasi data menggunakan Great Expectations.

=================================================

# **II. Membuat Data Context**

In [1]:
# Create a data context
from great_expectations.data_context import FileDataContext
import pandas as pd

context = FileDataContext.create(project_root_dir='./')

# **III. Konek ke Datasource**

In [7]:
df = pd.read_csv('dags/P2M3_alridho_data_clean.csv')
# Menghapus kolom dengan nama spesifik 'Unnamed: 0'
df = df.drop(columns=['Unnamed: 0'])
df.to_csv('P2M3_alridho_data_clean_gx.csv', index=False)

In [9]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'data-clean-store-sales-supermarket'
datasource = context.sources.add_pandas(datasource_name)
asset_name = 'data-store-sales'
path_to_data = 'P2M3_alridho_data_clean_gx.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)
# Build batch request
batch_request = asset.build_batch_request()

# **IV. Membuat Expectation Suite**

In [10]:
# Creat an expectation suite
expectation_suite_name = 'expectation-store-sales'
context.add_or_update_expectation_suite(expectation_suite_name)
# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)
# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5,total,date,time,payment,cogs,gross_margin_percentage,gross_income,rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.14,548.97,2019-01-05,13:08:00,Ewallet,522.83,4.76,26.14,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,2019-03-08,10:29:00,Cash,76.40,4.76,3.82,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.22,340.53,2019-03-03,13:23:00,Credit card,324.31,4.76,16.22,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.29,489.05,2019-01-27,20:33:00,Ewallet,465.76,4.76,23.29,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.21,634.38,2019-02-08,10:37:00,Ewallet,604.17,4.76,30.21,5.3


# **V. Expectations**

### **Expectation 1 (to be unique)**

Pada kolom `invoice_id` saya melakukan ekspektasi to be unique, dimana data dari invoice_id ini memang sudah seharusnya unique, karena 1 transaksi = 1 invoice. Jika ada invoice_id yang sama maka itu merupakan suatu eror dan harus di handle.

In [ ]:
# Expectation 1 : Column `invoice_id` must be unique

validator.expect_column_values_to_be_unique('invoice_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### **Expectation 2 (to be between min_value and max_value)**

Pada kolom `rating` saya melakukan ekspektasi to be between min_value and max_value, dimana data dari rating diharapkan memiliki nilai min_value = 0 dan max_value = 10. Karena pada saat pemberian rating memang range yang bisa di berikan oleh customer pada supermarket X adalah dari 0-10 tidak bisa lebih atau kurang.

In [22]:
# Expectation 2 : Column `rating` must be in range 0 to 10

validator.expect_column_values_to_be_between(column='rating', min_value=0, max_value=10)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### **Expectation 3 (to be in set)**

Pada kolom `product_line` saya melakukan ekspektasi to be in set, dimana data yang ada di dalam kolom ini harus merupakan salah satu dari 6 hal ini:
1. Health and beauty
2. Electronic accessories
3. Home and lifestyle
4. Sports and travel
5. Food and beverages
6. Fashion accessories

In [ ]:
# Expectation 3 : Column 'prduct_line' must contain one of the following 6 things.

validator.expect_column_values_to_be_in_set('product_line', ['Health and beauty','Electronic accessories',
        'Home and lifestyle', 'Sports and travel', 'Food and beverages','Fashion accessories'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 1000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### **Expectation 4 (to be in type list)**

Pada kolom `unit_price` saya melakukan ekspektasi to be in type list, dimana data pada kolom ini saya harapkan bertipe integer atau float, hal ini karena saya memiliki ekspektasi dimana harga dengan kurs dollar memang sudah seharusnya berbentuk integer atau float. Selain kedua tipe ini maka akan termasuk kedalam pelanggaran ekspektasi

In [28]:
# Expectation 4 : Column `unit_price` must in form of integer or float

validator.expect_column_values_to_be_in_type_list('unit_price', ['int64', 'float64'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### **Expectation 5 (table column count to equal)**

Pada `dataframe` saya melakukan ekspektasi table_column_count_to_equal, dimana saya memiliki ekspektasi bahwa dalam dataframe ini harus terdapat `17 column` yang meliputi:
1.    invoice_id: Identifikasi unik untuk setiap faktur atau transaksi.
2.    branch: Cabang atau lokasi tempat transaksi terjadi.
3.    city: Kota tempat cabang berada.
4.    customer Type: Menunjukkan apakah pelanggan adalah pelanggan tetap atau baru.
5.    gender: Jenis kelamin pelanggan.
6.    product Line: Kategori atau jenis produk yang dibeli.
7.    unit_price: Harga per unit produk.
8.    quantity: Jumlah unit produk yang dibeli.
9.    tax_5: Jumlah pajak (5% dari total biaya) yang diterapkan pada transaksi.
10.   total: Total biaya transaksi, termasuk pajak.
11.   date: Tanggal transaksi terjadi.
12.   time: Waktu saat transaksi terjadi.
13.   payment: Metode pembayaran yang digunakan (misalnya, kartu kredit, tunai).
14.   cogs: Biaya langsung terkait dengan produksi atau pembelian produk yang dijual.
15.   gross_margin_percentage: Persentase margin keuntungan untuk transaksi.
16.   gross_income: Total keuntungan yang diperoleh dari transaksi.
17.   rating: Penilaian atau umpan balik kepuasan pelanggan terhadap transaksi.

In [29]:
# Expectation 5 : Total Column dataframe must be 17

validator.expect_table_column_count_to_equal(value=17)

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 17
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### **Expectation 6 (column to exist)**

Saya melakukan ekspektasi dimana kolom `total` pada dataframe harus ada/exist. Saya menilai kolom ini sangat penting, karena mencakup harga total yang dibayar oleh customer. jika kolom ini tidak ada maka akan termasuk kedalam pelanggaran ekspektasi.

In [31]:
# Expectation 6 : Columns `total` must exist

validator.expect_column_to_exist(column='total')

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### **Expectation 7 (table row count to equal)**

Saya melakukan ekspektasi dimana pada tabel harus terdapat 1000 data. jika kurang atau lebih akan termasuk kedalam pelanggaran ekspektasi

In [36]:
# Expectation 7 : Total row in the dataframe must be 1000

validator.expect_table_row_count_to_equal(value=1000)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 1000
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### **Expectation 8 (column mean to be between)**

Saya melakukan ekspektasi column mean to be between pada kolom `gross_income`, untuk mengetahui apakah rata-rata gross_income ada di antara nilai minimum dan maximumnya. Nantinya informasi ini bisa berguna untuk mengetahui mana saja transaksi yang tergolong premium/di atas rata-rata gross_income.

In [37]:
min_gross = df['gross_income'].min()
max_gross = df['gross_income'].max()
validator.expect_column_mean_to_be_between('gross_income', min_value=min_gross, max_value=max_gross)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 15.380049999999999
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# **VI. Save Expectations**

In [38]:
# Save into Expectation Suite

validator.save_expectation_suite(discard_failed_expectations=False)

# **VII. Membuat Checkpoint**

In [39]:
# Create a checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

In [40]:
# Run a checkpoint

checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/26 [00:00<?, ?it/s]